In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
import time
from scipy.stats import t

In [2]:
plt.rcParams['figure.figsize'] = (15.0, 9.0)
plt.rcParams['figure.dpi'] = 100

In [3]:
f1 = pd.read_excel('./data/stability_demo.xlsx')
f1.head()

,date,day,sample,5E-S1-01,5E-S1-02,5E-S1-03,5E-S1-04,5E-S1-05,5E-H-01,5E-H-02,...,5E-S4-01,5E-S4-02,5E-S4-03,5E-S4-04,5E-S4-05,5E-Q1-01,5E-Q1-02,5E-Q1-03,5E-Q1-04,5E-Q1-05
0,2019-07-05,0,0M,8497.950,8547.034,8549.255,8554.364,8555.252,6271.153,6083.033,...,510.490,548.469,545.804,551.579,555.576,276.838,291.941,291.275,282.169,274.395
1,2019-07-19,14,1M,7863.262,7891.691,7899.667,7883.714,7888.009,5641.924,5884.080,...,518.795,539.861,564.608,591.810,589.560,307.726,308.339,283.387,311.816,274.797
2,2019-08-20,46,2M,7893.051,7851.237,8260.459,8260.459,8260.459,5945.639,5930.569,...,453.406,494.795,509.440,559.532,549.131,237.758,266.624,255.375,254.313,291.033
3,2019-09-23,80,3M,7422.638,7316.215,8243.970,8488.675,8119.019,6089.753,6107.151,...,509.668,519.158,520.514,524.807,569.771,407.086,341.109,327.552,388.784,353.988
4,2019-11-20,138,5M,8393.080,8283.950,8329.539,8553.232,8437.251,6435.824,6317.718,...,631.614,624.528,654.054,626.418,688.069,381.702,398.236,396.583,409.811,357.608


In [4]:
date = ['%s-%02d-%02d'%(i.year,i.month,i.day) for i in list(f1['date'])]
sample = list(f1['sample'])
day = list(f1['day'])

Col = ['5E-S1','5E-S2','5E-S3','5E-S4','5E-Q1','5E-Q2','5E-L','5E-H']
Suffix = ['01','02','03','04','05']

In [6]:
#Find aceeptable maximun date
LIST = [['Day']+day+['Slope','Y-intercept','SE Slope','Slope t-stat','p Value']]
for col in Col:
    List = []
    read = []
    for s in Suffix:
        read.append(list(f1[col+'-'+s]))

    mean = list(np.array(sum(np.matrix(read))/len(read))[0])   
    
    
    X = sm.add_constant(day)
    Y = mean
    model = sm.OLS(Y,X)
    results = model.fit()
    a0 = results.params[0]
    a1 = results.params[1]

    SSE = sum(results.resid**2)
    k = len(results.params)
    N = len(day)
    Syx2 = SSE/(N-k)
    Syx = np.sqrt(Syx2)
    t0975 = t.ppf(0.975,df=N-k)
    x_mean = np.mean(day)
    SSx = sum((np.array(day) - x_mean)**2)
    slope_p = results.pvalues[1]
    star = ''
    if (slope_p <= 0.05) & (slope_p > 0.01):
        star += '*'
    elif (slope_p <= 0.01) & (slope_p > 0.001):
        star += '**'        
    elif (slope_p <= 0.001):
        star += '***'          
    
    x_forline = np.arange(min(day),max(day),1)
    y_forline = a0 + x_forline*a1

    lower = y_forline-t0975*Syx*np.sqrt(1/N + (x_forline-x_mean)**2/SSx)
    upper = y_forline+t0975*Syx*np.sqrt(1/N + (x_forline-x_mean)**2/SSx)


    #find day0 value
    maxValue = mean[0]*1.1
    maxDay = sum(maxValue > upper)-1
    
    #表格
    List.append(col)
    List.extend(mean)
    List.append(a1)
    List.append(a0)
    List.append(results.bse[1])
    List.append(results.tvalues[1])
    List.append(slope_p)
    LIST.append(List)
    
    #plot
    #for r in range(len(read)):
    #    plt.plot(day,read[r],label=col+'-'+Suffix[r])
    fig, ax = plt.subplots(1, 1)
    ax.scatter(day,mean,label='mean',color='black')
    ax.plot(x_forline,y_forline,label='linear regression',color='black')
    ax.plot(x_forline,lower,label='95% C.I.',color='black',linestyle='--')
    ax.plot(x_forline,upper,color='black',linestyle='--')

    #plt.scatter(day,mean,label=col)
    #plt.plot(x_forline,y_forline,color='black')
    #plt.plot(x_forline,lower,color='black',linestyle='--')
    #plt.plot(x_forline,upper,color='black',linestyle='--')    
    
    ax.text(day[-4], min(lower),'slope=%.5f\np-value=%.5f%s'%(a1,slope_p,star),fontsize=18)
    day2 = day.copy()
    day2.append(maxDay)
    day2.sort()
    
    day3 = []
    for i in day2:
        if i == maxDay:
            day3.append('\n%s'%i)
        else:
            day3.append('%s'%i)
    
    plt.xticks(day2,day3,fontsize=14,rotation=0)
    plt.yticks(fontsize=14)
    plt.xlabel('Day',fontsize=16)
    plt.ylabel('Concentration',fontsize=16)
    plt.grid(linestyle='--')
    plt.legend(fontsize=12)
    plt.title(col,fontsize=20)
    ax.annotate('%.1f'%maxValue,[2,maxValue*1.005],color='red')
    
    plt.gca().get_xticklabels()[day2.index(maxDay)].set_color("red")
    
    
    ax.plot([0,maxDay],[maxValue,maxValue],color='red',linestyle=':')
    ax.scatter([0],[maxValue],color='red')
    ybound_lower,ybound_upper = ax.get_ybound()
    ax.plot([maxDay,maxDay],[ybound_lower,maxValue],color='red',linestyle=':')
    plt.ylim(ybound_lower,ybound_upper)
    plt.savefig('./results/%s.png'%col)
    plt.clf()
    #plt.show()

<Figure size 1500x900 with 0 Axes>

<Figure size 1500x900 with 0 Axes>

<Figure size 1500x900 with 0 Axes>

<Figure size 1500x900 with 0 Axes>

<Figure size 1500x900 with 0 Axes>

<Figure size 1500x900 with 0 Axes>

<Figure size 1500x900 with 0 Axes>

<Figure size 1500x900 with 0 Axes>

In [8]:
output = pd.DataFrame(np.asmatrix(LIST).T)
output.to_excel('./results/Statistical_Table.xlsx')